In [1]:
# |hide
from nblite import nbl_export

nbl_export();

In [2]:
from netrun_sim import (
    # Graph types
    Graph,
    Node,
    Edge,
    Port,
    PortRef,
    PortType,
    PortSlotSpec,
    PortState,
    SalvoCondition,
    SalvoConditionTerm,
    MaxSalvos,
    Salvo,
    # NetSim types
    NetSim,
    NetAction,
)

Define the graph

In [3]:
node_a = Node(
    name="A",
    out_ports={
        "out1": Port(PortSlotSpec.infinite()),
        "out2": Port(PortSlotSpec.finite(1)),
    },
    in_salvo_conditions={
        "manual": SalvoCondition(
            max_salvos=MaxSalvos.finite(1),
            ports=[],
            term=SalvoConditionTerm.true_(),
        ),
    },
    out_salvo_conditions={
        "1": SalvoCondition(
            max_salvos=MaxSalvos.finite(1),
            ports=["out1"],
            term=SalvoConditionTerm.port("out1", PortState.non_empty()),
        ),
        "2": SalvoCondition(
            max_salvos=MaxSalvos.finite(1),
            ports=["out2"],
            term=SalvoConditionTerm.port("out2", PortState.non_empty()),
        ),
    },
)

In [4]:
node_b1 = Node(
    name="B1",
    in_ports={"in": Port(PortSlotSpec.infinite())},
    out_ports={"out": Port(PortSlotSpec.infinite())},
    in_salvo_conditions={
        "default": SalvoCondition(
            max_salvos=MaxSalvos.finite(1),
            ports=["in"],
            term=SalvoConditionTerm.port("in", PortState.non_empty()),
        ),
    },
    out_salvo_conditions={
        "default": SalvoCondition(
            max_salvos=MaxSalvos.Infinite,
            ports=["out"],
            term=SalvoConditionTerm.port("out", PortState.non_empty()),
        ),
    },
)

In [5]:
node_b2 = Node(
    name="B2",
    in_ports={"in": Port(PortSlotSpec.infinite())},
    out_ports={"out": Port(PortSlotSpec.infinite())},
    in_salvo_conditions={
        "default": SalvoCondition(
            max_salvos=MaxSalvos.finite(1),
            ports=["in"],
            term=SalvoConditionTerm.port("in", PortState.non_empty()),
        ),
    },
    out_salvo_conditions={
        "default": SalvoCondition(
            max_salvos=MaxSalvos.Infinite,
            ports=["out"],
            term=SalvoConditionTerm.port("out", PortState.non_empty()),
        ),
    },
)

In [6]:
node_c = Node(
    name="C",
    in_ports={
        "in1": Port(PortSlotSpec.infinite()),
        "in2": Port(PortSlotSpec.infinite()),
    },
    out_ports={"out": Port(PortSlotSpec.infinite())},
)

In [7]:
edges = [
    Edge(
        PortRef("A", PortType.Output, "out1"),
        PortRef("B1", PortType.Input, "in"),
    ),
    Edge(
        PortRef("A", PortType.Output, "out2"),
        PortRef("B2", PortType.Input, "in"),
    ),
    Edge(
        PortRef("B1", PortType.Output, "out"),
        PortRef("C", PortType.Input, "in1"),
    ),
    Edge(
        PortRef("B2", PortType.Output, "out"),
        PortRef("C", PortType.Input, "in2"),
    ),
]

Create the graph and the net objects

In [8]:
graph = Graph([node_a, node_b1, node_b2, node_c], edges)
assert graph.validate() == []

net = NetSim(graph)

Try running the net until blocked. Should do nothing.

In [9]:
response_data, events = net.do_action(NetAction.run_net_until_blocked())
assert events == []

Create an epoch in node `A` manually

In [10]:
response_data, events = net.do_action(NetAction.create_epoch("A", Salvo("manual", [])))

Start the epoch inside node `A`

In [11]:
epoch_id = response_data.epoch.id
response_data, events = net.do_action(NetAction.start_epoch(epoch_id))

Create packets inside the node `A` epoch

In [12]:
response_data, events = net.do_action(
    NetAction.create_packet(
        epoch_id,
    )
)
packetA1_id = response_data.packet_id

response_data, events = net.do_action(
    NetAction.create_packet(
        epoch_id,
    )
)
packetA2_id = response_data.packet_id

Load packets into output ports in node `A`

In [13]:
response_data, events = net.do_action(
    NetAction.load_packet_into_output_port(packetA1_id, "out1")
)

response_data, events = net.do_action(NetAction.send_output_salvo(epoch_id, "1"))

response_data, events = net.do_action(
    NetAction.load_packet_into_output_port(packetA2_id, "out2")
)

response_data, events = net.do_action(NetAction.send_output_salvo(epoch_id, "2"))

Finish the epoch

In [14]:
response_data, events = net.do_action(NetAction.finish_epoch(epoch_id))

Run the net until blocked

In [15]:
response_data, events = net.do_action(NetAction.run_net_until_blocked())

print("Events in the net:")
for event in events:
    print("  ", event)

Events in the net:
   NetEvent.PacketMoved(ts=1768429486358, packet_id='01KEZ9S08AXFYD94QMNW2TG7RY', from=PacketLocation.edge(A.out.out2 -> B2.in.in), to=PacketLocation.input_port("B2", "in"))
   NetEvent.PacketMoved(ts=1768429486358, packet_id='01KEZ9S08A4B9DV22T6MWF1VWJ', from=PacketLocation.edge(A.out.out1 -> B1.in.in), to=PacketLocation.input_port("B1", "in"))
   NetEvent.InputSalvoTriggered(ts=1768429486358, epoch_id='01KEZ9S08PH3HRPWFVJB67KX22', condition="default")
   NetEvent.EpochCreated(ts=1768429486358, epoch_id='01KEZ9S08PH3HRPWFVJB67KX22')
   NetEvent.PacketMoved(ts=1768429486358, packet_id='01KEZ9S08A4B9DV22T6MWF1VWJ', from=PacketLocation.input_port("B1", "in"), to=PacketLocation.node('01KEZ9S08PH3HRPWFVJB67KX22'))
   NetEvent.InputSalvoTriggered(ts=1768429486358, epoch_id='01KEZ9S08PJXTCEFCVH7ZN3YYD', condition="default")
   NetEvent.EpochCreated(ts=1768429486358, epoch_id='01KEZ9S08PJXTCEFCVH7ZN3YYD')
   NetEvent.PacketMoved(ts=1768429486358, packet_id='01KEZ9S08AXFYD94

Should now have two startable epochs for `B1` and `B2`

In [16]:
startable_epochs = net.get_startable_epochs()
assert len(net.get_startable_epochs()) == 2
assert set([net.get_epoch(epoch_id).node_name for epoch_id in startable_epochs]) == {
    "B1",
    "B2",
}